In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional,GlobalAveragePooling1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import cv2
import numpy as np
import os
import glob
import json
from tensorflow import keras
from sklearn.model_selection import train_test_split

C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\google\api_core\_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.10). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\__init__.py:54: FutureWarning: 
    You are using a Python version 3.9 past its end of life. Google will update
    google-auth with critical bug fixes on a best-effort basis, but not
    with any other fixes or features. Please upgrade your Python version,
    and then update google-auth.
    
  warnings.warn(eol_message.format("3.9"), FutureWarning)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\google\oauth2\__init__.py:40: FutureWarni

In [2]:
# 1. Config
DATA_PATH        = 'Data'
LABEL_MAP_PATH   = 'Logs/label_map.json'
BATCH_SIZE       = 32
AUTOTUNE         = tf.data.AUTOTUNE
VAL_SPLIT        = 0.1
TEST_SPLIT       = 0.1

# 2. Load label_map from JSON
with open(LABEL_MAP_PATH, 'r', encoding='utf-8') as f:
    label_map = json.load(f)

NUM_CLASSES = len(label_map)
print(f"Number of classes: {NUM_CLASSES}")

# 3. List all .npz files (Data/<action_name>/*.npz)
file_pattern = os.path.join(DATA_PATH, '**', '*.npz')
all_files = glob.glob(file_pattern, recursive=True)
print(f"Found {len(all_files)} samples.")

# Stratify by class (folder name) so train/val/test have similar class distribution
stratify_labels = [os.path.basename(os.path.dirname(p)) for p in all_files]
train_files, temp_files = train_test_split(
    all_files,
    test_size=VAL_SPLIT + TEST_SPLIT,
    shuffle=True,
    random_state=42,
    stratify=stratify_labels
)

stratify_temp = [os.path.basename(os.path.dirname(p)) for p in temp_files]
val_files, test_files = train_test_split(
    temp_files,
    test_size=TEST_SPLIT / (VAL_SPLIT + TEST_SPLIT),
    shuffle=True,
    random_state=42,
    stratify=stratify_temp
)

print(f"  Train samples: {len(train_files)}")
print(f"    Val samples: {len(val_files)}")
print(f"   Test samples: {len(test_files)}")

# 4. Parse each .npz file
def _load_npz(path):
    npz_path = path.decode('utf-8')
    data = np.load(npz_path)
    seq = data['sequence'].astype(np.float32)
    lbl = np.int32(data['label'])
    return seq, lbl

def parse_fn(path):
    seq, lbl = tf.numpy_function(
        func=_load_npz,
        inp=[path],
        Tout=[tf.float32, tf.int32]
    )
    seq.set_shape([60, 201])
    lbl.set_shape([])
    return seq, lbl
def make_dataset(file_list, shuffle=False, repeat=False):
    ds = tf.data.Dataset.from_tensor_slices(file_list)
    if shuffle:
        ds = ds.shuffle(len(file_list), reshuffle_each_iteration=True)
    if repeat:
        ds = ds.repeat()
    ds = ds.map(parse_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True)
    ds = ds.prefetch(AUTOTUNE)
    return ds

# 6. Tạo train_ds & val_ds
train_ds = make_dataset(train_files, shuffle=True, repeat=True)
val_ds   = make_dataset(val_files, shuffle=False, repeat=False)
test_ds  = make_dataset(test_files, shuffle=False, repeat=False)

# 7. Compute steps
steps_per_epoch = len(train_files) // BATCH_SIZE
validation_steps = len(val_files) // BATCH_SIZE

Found 5049 samples.
  Train samples: 4039
    Val samples: 505
   Test samples: 505


In [3]:
# When number of classes grows (100 -> 500 -> 1000+), use larger model for better discrimination.
# Below 500 classes: standard size; 500+: large model.
USE_LARGE_MODEL = NUM_CLASSES >= 500
lstm_units = 384 if USE_LARGE_MODEL else 256
dense_1, dense_2 = (768, 384) if USE_LARGE_MODEL else (512, 256)
if USE_LARGE_MODEL:
    print("Using LARGE model (LSTM 384, Dense 768->384) for many classes.")

inputs = tf.keras.Input(shape=(60, 201))

# LSTM blocks
x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=0.3))(inputs)
x = BatchNormalization()(x)
x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=0.3))(x)
x = BatchNormalization()(x)
x = Bidirectional(LSTM(lstm_units, dropout=0.3))(x)
x = BatchNormalization()(x)

# Dense layers
x = Dense(dense_1, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(dense_2, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

# Output layer: number of units = number of classes (from label_map)
outputs = Dense(NUM_CLASSES, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Label smoothing helps when there are many classes (less overconfident, better generalization)
LABEL_SMOOTHING = 0.1
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(label_smoothing=LABEL_SMOOTHING),
    metrics=['accuracy']
)

In [4]:
# 1. Checkpoint directory
checkpoint_dir = 'Models/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'final_model.keras')

# 2. Callbacks
callbacks = [
    ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

In [ ]:
model.fit(
    train_ds,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks = callbacks
)

Epoch 1/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0355 - loss: 7.1225
Epoch 1: val_loss improved from inf to 3.77955, saving model to Models/checkpoints\final_model.keras
126/126 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.0357 - loss: 7.1121 - val_accuracy: 0.1125 - val_loss: 3.7796
Epoch 2/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.1600 - loss: 3.0988
Epoch 3/100


C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2868 - loss: 2.3785
Epoch 3: val_loss improved from 3.77955 to 1.79479, saving model to Models/checkpoints\final_model.keras
126/126 ━━━━━━━━━━━━━━━━━━━━ 259s 2s/step - accuracy: 0.2870 - loss: 2.3777 - val_accuracy: 0.4271 - val_loss: 1.7948
Epoch 4/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 230s 2s/step - accuracy: 0.3870 - loss: 1.9422
Epoch 5/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4507 - loss: 1.6407
Epoch 5: val_loss improved from 1.79479 to 1.57072, saving model to Models/checkpoints\final_model.keras
126/126 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.4508 - loss: 1.6400 - val_accuracy: 0.4875 - val_loss: 1.5707
Epoch 6/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 304s 2s/step - accuracy: 0.5283 - loss: 1.3803
Epoch 7/100
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5891 - loss: 1.1196
Epoch 7: val_loss improved from 1.57072 to 0.70064, saving model to Models/checkpoints\final_model.keras
126/126 ━━━━━━━━━━━━━━━━━━━